In [1]:
from oauth import *
email_address = "indecorpredicate@gmail.com"

In [2]:
MyGmail = OAuthGmailUser(email_address, cred_path=CRED_PATH, token_path=TOKEN_PATH, scopes=["modify"])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=467488332884-nkn4j4q35qgq66vterij95d56limmhn5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54029%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=A7EfklqzJRT3hVvCvKliGONfBl8nRh&access_type=offline
starting a new service


In [3]:
my_messages = MyGmail.get_messages(max_results=4)

In [23]:
import time
from base64 import urlsafe_b64encode

In [252]:
class GmailMessage(email.message.EmailMessage):
    def __init__(self, user, policy=email.policy.SMTP):
        super().__init__(policy)
        self.user = user

        # API Atributes
        self.internalDate = time.time()*1000
        self.historyId = None
        # a big one 
        self.payload = None
        self.snippet = self.get_body(('plain')).as_string()[:58]
        self.raw = None
        self.sizeEstimate = None
        self.threadId = None
        self.labelIds = None
        self.id = None

    def set_api_attrs(self):
        """Refreshes and/or sets the API attributes based on current content."""
        self.internalDate = time.time()*1000
        # self.historyId = None
        # a big one 
        self.payload = self.get_gmail_payload()
        self.snippet = self.get_body(('plain')).as_string()[:58]
        self.raw = urlsafe_b64encode(self.as_bytes())
        # self.sizeEstimate = None
        # self.threadId = None
        # self.labelIds = None
        # self.id = None
    
    def get_gmail_payload(self):
        payload = {
            'body': { # The body of a single MIME message part. # The message part body for this part, which may be empty for container MIME message parts.
                "data": urlsafe_b64encode(self.as_bytes()), # The body data of a MIME message part as a base64url encoded string. May be empty for MIME container types that have no message body or when the body data is sent as a separate attachment. An attachment ID is present if the body data is contained in a separate attachment.
                # "attachmentId": "A String", # When present, contains the ID of an external attachment that can be retrieved in a separate messages.attachments.get request. When not present, the entire content of the message part body is contained in the data field.
                # "size": 42, # Number of bytes for the message part data (encoding notwithstanding).
            },
            "mimeType": self['Content-Type'], # The MIME type of the message part.
            # "partId": "A String", # The immutable ID of the message part.
            # "filename": "A String", # The filename of the attachment. Only present if this message part represents an attachment.
            "headers":[
                dict(self.items())
            ]
            # "parts": [ # The child MIME message parts of this part. This only applies to container MIME message parts, for example multipart/*. For non- container MIME message part types, such as text/plain, this field is empty. For more information, see RFC 1521.

        }
        # "payload": { # A single MIME message part. # The parsed email structure in the message parts.
        #     "body": { # The body of a single MIME message part. # The message part body for this part, which may be empty for container MIME message parts.
        #     "data": "A String", # The body data of a MIME message part as a base64url encoded string. May be empty for MIME container types that have no message body or when the body data is sent as a separate attachment. An attachment ID is present if the body data is contained in a separate attachment.
        #     "attachmentId": "A String", # When present, contains the ID of an external attachment that can be retrieved in a separate messages.attachments.get request. When not present, the entire content of the message part body is contained in the data field.
        #     "size": 42, # Number of bytes for the message part data (encoding notwithstanding).
        #         },
        #     "mimeType": "A String", # The MIME type of the message part.
        #     "partId": "A String", # The immutable ID of the message part.
        #     "filename": "A String", # The filename of the attachment. Only present if this message part represents an attachment.
        #     "headers": [ # List of headers on this message part. For the top-level message part, representing the entire message payload, it will contain the standard RFC 2822 email headers such as To, From, and Subject.
        #         {"name": "A String", # The name of the header before the : separator. For example, To.
        #         "value": "A String", # The value of the header after the : separator. For example, someuser@example.com.},
        #         ],
        #     "parts": [ # The child MIME message parts of this part. This only applies to container MIME message parts, for example multipart/*. For non- container MIME message part types, such as text/plain, this field is empty. For more information, see RFC 1521.
        #         # Object with schema name: MessagePart],
        # },
        return payload


    def as_api_consumable(self, api='gmail_v1', refresh=True):
        if refresh:
            self.set_api_attrs()
        consumable_draft = {
            # #an email message.
            # "internalDate": self.internalDate, # The internal message creation timestamp (epoch ms), which determines ordering in the inbox. For normal SMTP-received email, this represents the time the message was originally accepted by Google, which is more reliable than the Date header. However, for API-migrated mail, it can be configured by client to be based on the Date header.
            # "historyId": self.historyId, # The ID of the last history record that modified this message.
            "payload": self.payload, # A single MIME message part. # The parsed email structure in the message parts.
            "snippet": self.snippet, # A short part of the message text.
            # "raw": self.raw, # The entire email message in an RFC 2822 formatted and base64url encoded string. Returned in messages.get and drafts.get responses when the format=RAW parameter is supplied.
            # "sizeEstimate": self.sizeEstimate, # Estimated size in bytes of the message.
            "threadId": self.threadId, # The ID of the thread the message belongs to. To add a message or draft to a thread, the following criteria must be met:
                # - The requested threadId must be specified on the Message or Draft.Message you supply with your request.
                # - The References and In-Reply-To headers must be set in compliance with the RFC 2822 standard.
            #     # - The Subject headers must match.
            # "labelIds": self.labelIds, # List of IDs of labels applied to this message.
            # "id": self.id, # The immutable ID of the message.
        }
        api_consumable = dict()
        for attr, data in consumable_draft.items():
            if data:
                api_consumable[attr] = data
        return api_consumable


        
    def add_sole_header(self, _name, _value, **_params):
        del self[_name]
        self.add_header(_name, _value, **_params)

    def add_sole_headers(self, **headers):
        for name, value in headers.items():
            self.add_sole_header(name, value)
    
    def send(self):
        self.user._send_message(self.as_api_consumable())

    


In [253]:
class UpgradedGmailUser(OAuthGmailUser):
    def new_message(self, to: str, subject, contents):
        NewMessage = GmailMessage(self)
        NewMessage.add_sole_headers(To=to,From=self.user_id, Subject=subject)
        NewMessage.set_content(contents)
        return NewMessage
    
    def _send_message(self, message, media_upload=None):
        self._messages_api().send(userId=self.user_id, body=message, media_body=None, media_mime_type=None).execute()
    
    def new_reply(self, reply_to:GMessage, contents, subject=None):
        if not subject:
            subject = reply_to.mime["Subject"]
            if "Re: " != subject[:4]:
                subject = "Re: " + subject
        message = self.new_message(to=reply_to.mime["From"], subject = subject, contents=contents)
        message.threadId = reply_to.threadId
        reply_id = reply_to.mime["Message-ID"]
        message.add_sole_header("In-Reply-To",reply_id)
        references = reply_to.mime["References"]
        if not references:
            references = reply_to.mime["In-Reply-To"]
        references += f"\t{reply_id}"
        message.add_sole_header("References", references)
        return message

In [254]:
4!=5

True

In [255]:
MyGmail = UpgradedGmailUser(email_address, cred_path=CRED_PATH, token_path=TOKEN_PATH, scopes=["modify"])
reply_content = "Hi,  I'm a script that's testing API integration.  did you get this?  thanks!"
my_rep = MyGmail.new_reply(mes, reply_content)



starting a new service


In [256]:
my_rep.send()

TypeError: Object of type 'bytes' is not JSON serializable

In [133]:

MyGMessage.set_content("Hi,  I'm a script that's testing API integration.  did you get this?  thanks!")

MyGMessage.add_sole_header("To", "ben@terminallabs.com")
MyGMessage.add_sole_header("From", MyGMessage.user.user_id)
MyGMessage.add_sole_header("Subject", "Testing a Bot")

To, ben@terminallabs.com, {}
From, indecorpredicate@gmail.com, {}
Subject, Testing a Bot, {}


In [123]:
from pprint import pprint
pprint(dict(MyGMessage.items()))
pprint(dict(MyGMessage))

{'Content-Transfer-Encoding': '7bit',
 'Content-Type': 'text/plain; charset="utf-8"',
 'From': 'indecorpredicate@gmail.com',
 'MIME-Version': '1.0',
 'Subject': 'Testing a Bot',
 'To': 'ben@terminallabs.com'}
{'Content-Transfer-Encoding': '7bit',
 'Content-Type': 'text/plain; charset="utf-8"',
 'From': 'indecorpredicate@gmail.com',
 'MIME-Version': '1.0',
 'Subject': 'Testing a Bot',
 'To': 'ben@terminallabs.com'}


In [111]:
MyGmail._messages_api().get(userId=MyGmail.user_id, id='17b7e9da190bdf5b').execute()

{'id': '17b7e9da190bdf5b',
 'threadId': '17b7e9d38e2a9fe8',
 'labelIds': ['UNREAD', 'IMPORTANT', 'SENT', 'INBOX'],
 'snippet': 'Hi, I&#39;m a body PaRagRaPh @!SDXCV^',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'MIME-Version', 'value': '1.0'},
   {'name': 'Date', 'value': 'Wed, 25 Aug 2021 14:41:04 -0400'},
   {'name': 'Message-ID',
    'value': '<CAFGEsPv1hWe3uBCi3LXwWhovm7WMauc9MCJdibw=qXhUOBtR7Q@mail.gmail.com>'},
   {'name': 'Subject', 'value': "I'm a Subject LiNe"},
   {'name': 'From', 'value': 'Indecor Predicate <indecorpredicate@gmail.com>'},
   {'name': 'To', 'value': 'Indecor Predicate <indecorpredicate@gmail.com>'},
   {'name': 'Content-Type',
    'value': 'multipart/alternative; boundary="000000000000bcb40b05ca6697c6"'}],
  'body': {'size': 0},
  'parts': [{'partId': '0',
    'mimeType': 'text/plain',
    'filename': '',
    'headers': [{'name': 'Content-Type',
      'value': 'text/plain; charset="UTF-8"'}],
   

In [68]:
with open('text.txt') as fp:
    fp.read()

In [71]:
type(open('text.txt').read())

str

In [74]:
mes = email.message.EmailMessage()
mes.set_content("this is some content")

In [77]:
mes.get_body().as_string()

'Content-Type: text/plain; charset="utf-8"\nContent-Transfer-Encoding: 7bit\nMIME-Version: 1.0\n\nthis is some content\n'

In [79]:
mes.as_string()

'Content-Type: text/plain; charset="utf-8"\nContent-Transfer-Encoding: 7bit\nMIME-Version: 1.0\n\nthis is some content\n'

In [127]:
d = {"key": "val", 1:2}
# {key: val for key, val in d}
(*d)


SyntaxError: can't use starred expression here (<ipython-input-127-50b70f771192>, line 6)

In [129]:
if True: print(3)

3


In [165]:
mes = MyGmail.get_messages()[0]

In [168]:
mes.mime["Subject"]

"Re: I'm a Subject LiNe"

In [164]:
mes.mime["In-Reply-To"]

In [170]:
"Re: " == [:4]

True

In [180]:
def kwargs(**kwargs):
    print(kwargs)
kwargs(this="that")

{'this': 'that'}
